In [0]:
import requests
from databricks.sdk import WorkspaceClient
import io

In [0]:
workspace_client = WorkspaceClient()
host = workspace_client.config.host
print(host)

In [0]:
headers = {'Authorization' : 'Bearer {}'.format(dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get())}

In [0]:
space_id = dbutils.widgets.get("genie-space-id")
print(space_id)

In [0]:
resp = requests.get(f"{host}/api/2.0/genie/spaces/{space_id}?include_serialized_space=true",
                    headers=headers)
export_payload = resp.json()
print(export_payload)

In [0]:
serialized_space = export_payload["serialized_space"]
print(serialized_space)


In [0]:
catalog = "dkushari_uc"
schema = "starbucks_genie"
volume = "exp_imp"

In [0]:
spark.sql(f"create volume if not exists {catalog}.{schema}.{volume};")

In [0]:
file_name = "serialized_space.json"
volume_file_path = f"/Volumes/{catalog}/{schema}/{volume}/{file_name}"

In [0]:
workspace_client.files.create_directory(f"/Volumes/{catalog}/{schema}/{volume}")
data_bytes = io.BytesIO(serialized_space.encode("utf-8"))
workspace_client.files.upload(volume_file_path, data_bytes, overwrite=True)